# Score: 52.28%

### The trick to this competition is how to handle huge amounts of data.

- Forums seem to say the best way of approaching this type of problem is by loading the data in chunks

### It also appears that the predictor variable is very rare

- It's usually the case that the value is zero, I guess we'll supposed to fit to the few cases where it isn't

In [2]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [19]:
training_data = pd.read_csv('../data/talking_data/train.csv', nrows=10000)
training_data.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

In [23]:
training_data.head(10)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
5,18787,3,1,16,379,2017-11-06 14:36:26,NaN,0
6,103022,3,1,23,379,2017-11-06 14:37:44,NaN,0
7,114221,3,1,19,379,2017-11-06 14:37:59,NaN,0
8,165970,3,1,13,379,2017-11-06 14:38:10,NaN,0
9,74544,64,1,22,459,2017-11-06 14:38:23,NaN,0


In [24]:
predictor_cols = ['app', 'device', 'os', 'channel']

train_y = training_data.is_attributed
train_x = training_data[predictor_cols]
ohe_train_x = pd.get_dummies(train_x)
fitting_x, val_x, fitting_y, val_y = train_test_split(ohe_train_x.as_matrix(), train_y.as_matrix(), test_size=0.25)

imputer = Imputer()
imp_fitting_x = imputer.fit_transform(fitting_x)
imp_val_x = imputer.transform(val_x)

In [25]:
model = XGBRegressor()
model.fit(imp_fitting_x, fitting_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [26]:
predictions = model.predict(val_x)

print("Mean Absolute Error : " + str(mean_absolute_error(predictions, val_y)))

Mean Absolute Error : 0.002950698220729828


In [27]:
testing_data = pd.read_csv('../data/talking_data/test.csv')
test_x = testing_data[predictor_cols]
ohe_test_x = pd.get_dummies(test_x)
ohe_test_x

,app,device,os,channel
0,9,1,3,107
1,9,1,3,466
2,21,1,19,128
3,15,1,13,111
4,12,1,13,328
5,18,1,13,107
6,3,1,1,137
7,27,1,19,153
8,18,1,10,107
9,12,1,53,424


In [15]:
submission = model.predict(ohe_test_x.as_matrix())

mod_submission = list()
for i in range(len(submission)):
    result = submission[i]
    if result < 0.5:
        mod_submission.append(0)
    else:
        mod_submission.append(1)

In [16]:
submission[:5]

array([ 6.0349703e-05, -2.8407574e-04,  1.2354553e-03,  6.0349703e-05,
       -2.8407574e-04], dtype=float32)

In [14]:
import os
os.sys.path.append("/Users/pbezuhov/git/custom_libraries/")
import main

columns = ["click_id", "is_attributed"]
ids = test['click_id'].as_matrix()
data = list(zip(ids, list(submission)))
main.save_file('/Users/pbezuhov/git/Kaggle/submissions/talking_data/1_partial_data_10000_rows.csv', data, columns)